In [ ]:
import scipy.io
import jax.numpy as np
from scipy.io import loadmat 
import numpy as onp
import matplotlib.pyplot as plt 
import os
import shutil
from PIL import Image
from inrmri.bart import bart_acquisition_from_arrays
from inrmri.data_harvard import get_csmaps_and_mask, get_reference_reco
import pandas as pd
from jax import jit, vmap, random
from inrmri.dip import TimeDependant_DIP_Net, helix_generator, circle_generator
from inrmri.new_radon import ForwardRadonOperator
from inrmri.fourier import fastshiftfourier, get_freqs
from inrmri.basic_nn import weighted_loss 
from inrmri.utils import to_complex, is_inside_of_radial_lim, meshgrid_from_subdiv_autolims, total_variation_batch_complex, save_matrix_and_dict_in_zpy, load_matrix_and_dict_from_zpy    
from inrmri.utils import create_exp_file_name, total_variation_complex
import optax 
from inrmri.metrics_rd import mean_psnr, mean_ssim, mean_artifact_power
from inrmri.utils_rdls import seconds_to_min_sec_format, filter_and_get_columns, apply_transform

In [ ]:
from inrmri.image_processor import BeforeLinRegNormalizer
from inrmri.basic_plotting import full_halph_FOV_space_time 
from inrmri.image_processor import reduce_FOV 

In [ ]:
from inrmri.utils_rdls import safe_normalize, get_center

## General

In [ ]:
total_slices = 8
num_frames   = 30
slice_num             = 7
base_path    = '/mnt/workspace/datasets/pulseqCINE/'
stdip_parameters = {
    'experiment_name': 'total_kiter_lambda_denoise_reg', 
    'training_params': {
        'total_kiter':3000,
        'lambda_denoise_reg':1e-7,
        'slice':slice_num,
    }
}
tddip_parameters = {
    'experiment_name': 'total_kiter', 
    'training_params': {
        'total_kiter':2000,
        'slice':slice_num,
    }
}
# parameteres for global statustics
stdip_summray_parameters = stdip_parameters['training_params']
stdip_summray_parameters.pop('slice', None)
tddip_summray_parameters = tddip_parameters['training_params']
tddip_summray_parameters.pop('slice', None)

In [ ]:
dict_plot = {
    'DATA_1.5T': {
        'GC': {
            1: {'crop_ns': [35,50,55,30], 'saturation': 0.25},
            2: {'crop_ns': [35,50,55,30], 'saturation': 0.25}, # done
            3: {'crop_ns': [32,57,55,27], 'saturation': 0.3}, # done
            4: {'crop_ns': [32,47,57,27], 'saturation': 0.4}, # done
            5: {'crop_ns': [25,55,50,30], 'saturation': 0.5}, # done
            6: {'crop_ns': [23,55,48,30], 'saturation': 0.5}, # done
            7: {'crop_ns': [25,55,50,30], 'saturation': 0.5},
            8: {'crop_ns': [25,55,50,30], 'saturation': 0.5},
        },
        'CM': {
            1: {'crop_ns': [32,43,30,40], 'saturation': 0.4},
            2: {'crop_ns': [32,43,30,40], 'saturation': 0.4},
            3: {'crop_ns': [32,43,30,40], 'saturation': 0.4}, # done
            4: {'crop_ns': [35,40,35,40], 'saturation': 0.4}, # done
            5: {'crop_ns': [35,40,35,40], 'saturation': 0.4}, # done
            6: {'crop_ns': [35,40,40,34], 'saturation': 0.4}, # done
            7: {'crop_ns': [35,40,35,40], 'saturation': 0.4}, # done
            8: {'crop_ns': [35,40,35,40], 'saturation': 0.4},
        },
        'DW': {
            1: {'crop_ns': [28,45,50,25], 'saturation': 0.6},
            2: {'crop_ns': [28,45,50,25], 'saturation': 0.6}, # done
            3: {'crop_ns': [28,45,50,25], 'saturation': 0.6}, # done
            4: {'crop_ns': [28,45,50,25], 'saturation': 0.6}, # done
            5: {'crop_ns': [30,45,50,25], 'saturation': 0.6}, # done
            6: {'crop_ns': [30,45,50,25], 'saturation': 0.6},
            7: {'crop_ns': [30,45,50,25], 'saturation': 0.6}, 
            8: {'crop_ns': [30,45,50,25], 'saturation': 0.6},
        },
    },
    'DATA_0.55T': {
        'MP': {
            1: {'crop_ns': [40,35,35,40], 'saturation': 0.3}, # done
            2: {'crop_ns': [40,35,35,40], 'saturation': 0.3}, # done
            3: {'crop_ns': [40,35,35,40], 'saturation': 0.3}, # done
            4: {'crop_ns': [40,35,35,40], 'saturation': 0.3}, # done
            5: {'crop_ns': [40,40,40,40], 'saturation': 0.3}, # done
            6: {'crop_ns': [40,40,40,40], 'saturation': 0.3},
            7: {'crop_ns': [40,40,40,40], 'saturation': 0.3},
            8: {'crop_ns': [40,40,40,40], 'saturation': 0.3},
        },
        'FH': {
            1: {'crop_ns': [32,40,35,35], 'saturation': 0.3}, 
            2: {'crop_ns': [32,40,35,35], 'saturation': 0.3}, # done
            3: {'crop_ns': [32,40,33,38], 'saturation': 0.3}, # done
            4: {'crop_ns': [32,40,33,38], 'saturation': 0.3}, # done
            5: {'crop_ns': [32,40,33,38], 'saturation': 0.3}, # done
            6: {'crop_ns': [32,40,33,38], 'saturation': 0.3}, # done
            7: {'crop_ns': [32,40,33,38], 'saturation': 0.3},
            8: {'crop_ns': [32,40,33,38], 'saturation': 0.3},
        }, 
        'TC': {
            1: {'crop_ns': [32,42,48,37], 'saturation': 0.2},
            2: {'crop_ns': [40,40,45,40], 'saturation': 0.2}, # done
            3: {'crop_ns': [40,40,45,40], 'saturation': 0.2}, # done
            4: {'crop_ns': [40,40,45,40], 'saturation': 0.2}, # done
            5: {'crop_ns': [40,45,45,40], 'saturation': 0.2}, # done
            6: {'crop_ns': [40,45,45,40], 'saturation': 0.25}, # done
            7: {'crop_ns': [40,45,45,40], 'saturation': 0.25},
            8: {'crop_ns': [40,45,45,40], 'saturation': 0.25},
        }, 
        'RF': {
            1: {'crop_ns': [38,42,45,40], 'saturation': 0.2},
            2: {'crop_ns': [38,42,45,40], 'saturation': 0.2}, # done
            3: {'crop_ns': [38,42,45,40], 'saturation': 0.2}, # done
            4: {'crop_ns': [38,42,45,40], 'saturation': 0.25}, # done
            5: {'crop_ns': [38,42,45,40], 'saturation': 0.25}, # done
            6: {'crop_ns': [38,42,45,40], 'saturation': 0.25}, # done
            7: {'crop_ns': [38,42,45,40], 'saturation': 0.25},
            8: {'crop_ns': [38,42,45,40], 'saturation': 0.25},
        }, 
        'JB': {
            1: {'crop_ns': [40,38,48,30], 'saturation': 0.4},
            2: {'crop_ns': [40,38,48,30], 'saturation': 0.4}, # done
            3: {'crop_ns': [40,38,48,30], 'saturation': 0.5}, # done
            4: {'crop_ns': [40,38,48,30], 'saturation': 0.5}, # done
            5: {'crop_ns': [40,38,48,30], 'saturation': 0.5}, # done
            6: {'crop_ns': [40,38,48,30], 'saturation': 0.5}, # done
            7: {'crop_ns': [40,38,48,30], 'saturation': 0.5},
            8: {'crop_ns': [40,38,48,30], 'saturation': 0.5},
        }, 
        'FE': {
            1: {'crop_ns': [40,40,42,38], 'saturation': 0.25},
            2: {'crop_ns': [40,40,42,38], 'saturation': 0.25}, # done
            3: {'crop_ns': [40,40,42,38], 'saturation': 0.25}, # done
            4: {'crop_ns': [38,42,38,36], 'saturation': 0.25}, # done
            5: {'crop_ns': [36,42,38,36], 'saturation': 0.35}, # done 
            6: {'crop_ns': [36,42,38,36], 'saturation': 0.35}, # done 
            7: {'crop_ns': [36,42,38,36], 'saturation': 0.4}, # done
            8: {'crop_ns': [36,42,38,36], 'saturation': 0.5}, # done
        }, 
        'DB': {
            1: {'crop_ns': [34,40,45,30], 'saturation': 0.25},
            2: {'crop_ns': [34,40,45,30], 'saturation': 0.25}, # done
            3: {'crop_ns': [34,40,45,30], 'saturation': 0.25}, # done
            4: {'crop_ns': [34,40,45,30], 'saturation': 0.25}, # done
            5: {'crop_ns': [34,40,45,30], 'saturation': 0.25}, # done
            6: {'crop_ns': [34,42,45,30], 'saturation': 0.25}, # done
            7: {'crop_ns': [34,42,45,30], 'saturation': 0.25},
            8: {'crop_ns': [34,42,45,30], 'saturation': 0.25},
        }, 
        'FP': {
            1: {'crop_ns': [35,42,45,30], 'saturation': 0.35},
            2: {'crop_ns': [35,42,45,30], 'saturation': 0.35}, # done
            3: {'crop_ns': [35,42,45,30], 'saturation': 0.35}, # done
            4: {'crop_ns': [35,42,45,30], 'saturation': 0.35}, # done
            5: {'crop_ns': [35,42,45,30], 'saturation': 0.35},
            6: {'crop_ns': [35,42,45,30], 'saturation': 0.35},
            7: {'crop_ns': [35,42,45,30], 'saturation': 0.35}, # done
            8: {'crop_ns': [35,42,45,30], 'saturation': 0.35},
        }, 
        'AA': {
            1: {'crop_ns': [35,42,45,30], 'saturation': 0.3},
            2: {'crop_ns': [35,42,45,30], 'saturation': 0.3}, # done
            3: {'crop_ns': [35,42,45,30], 'saturation': 0.3}, # done
            4: {'crop_ns': [35,42,45,30], 'saturation': 0.3}, # done
            5: {'crop_ns': [35,42,45,30], 'saturation': 0.3}, # done
            6: {'crop_ns': [35,42,45,30], 'saturation': 0.3}, # done
            7: {'crop_ns': [35,42,45,30], 'saturation': 0.3},
            8: {'crop_ns': [35,42,45,30], 'saturation': 0.3},
        }, 
        'DP': {
            1: {'crop_ns': [25,60,40,35], 'saturation': 0.2},
            2: {'crop_ns': [25,60,40,35], 'saturation': 0.2}, # done
            3: {'crop_ns': [25,55,40,35], 'saturation': 0.2}, # done
            4: {'crop_ns': [25,55,40,35], 'saturation': 0.2}, # done
            5: {'crop_ns': [25,55,40,35], 'saturation': 0.2}, # done
            6: {'crop_ns': [25,55,40,35], 'saturation': 0.2}, # done
            7: {'crop_ns': [25,55,40,40], 'saturation': 0.2}, # done
            8: {'crop_ns': [25,55,40,40], 'saturation': 0.2},
        }, 
    }
}

dict_trans = {
    'DATA_1.5T': {
        'GC': {
            'flip_h': False,
            'flip_v': False,
            'rot': 0,
        },
        'CM': {
            'flip_h': False,
            'flip_v': False,
            'rot': 0,
        },
        'DW': {
            'flip_h': False,
            'flip_v': False,
            'rot': 0,
        },
    },
    'DATA_0.55T': {
        'MP': {
            'flip_h': True,
            'flip_v': True,
            'rot': 0,
        },
        'FH': {
            'flip_h': True,
            'flip_v': True,
            'rot': 0,
        }, 
        'TC': {
            'flip_h': True,
            'flip_v': True,
            'rot': 0,
        }, 
        'RF': {
            'flip_h': True,
            'flip_v': True,
            'rot': 0,
        }, 
        'JB': {
            'flip_h': True,
            'flip_v': True,
            'rot': 0,
        }, 
        'FE': {
            'flip_h': True,
            'flip_v': False,
            'rot': 0,
        }, 
        'DB': {
            'flip_h': True,
            'flip_v': False,
            'rot': 0,
        },
        'FP': {
            'flip_h': False,
            'flip_v': False,
            'rot': 0,
        },
        'AA': {
            'flip_h': True,
            'flip_v': False,
            'rot': 0,
        },
        'DP': {
            'flip_h': False,
            'flip_v': False,
            'rot': 0,
        },
    }
}


gt_trans = {
    'DATA_0.55T': {
        'FP': {
            'flip_h': False,
            'flip_v': False,
            'rot': 90,
        },
        'DP': {
            'flip_h': True,
            'flip_v': True,
            'rot': 0,
        },
    }
}


# 1 .- Define volunteer and experiment

In [ ]:
volunteer    = 'TC'
dataset      = 'DATA_0.55T'
# --- PATH ---
base_folder                = base_path + dataset + '/' + volunteer + '/'
train_data_folder          = base_folder + 'traindata/'

## 1.1 .- Ground truth data

In [ ]:
total_time_grasp = 0
total_time_sense = 0
for i in range(total_slices):
    slice_item = i+1
    dataset_name       = 'slice_' + str(slice_item) + '_' + str(total_slices) +'_nbins' + str(num_frames)
    path_dataset       = train_data_folder + dataset_name + '.npz'
    data               = onp.load(path_dataset)
    total_time_grasp   += data["time_grasp"]
    total_time_sense   += data["time_sense"]
total_time_grasp, total_time_sense

In [ ]:
dataset_name          = 'slice_' + str(slice_num) + '_' + str(total_slices) +'_nbins' + str(num_frames)
path_dataset          = train_data_folder + dataset_name + '.npz'

In [ ]:
data         = onp.load(path_dataset)
recon_grasp  = data["recon_grasp"]
recon_sense  = data["recon_sense"]
recon_fs     = data["recon_fs"]
time_grasp   = data["time_grasp"]
time_sense   = data["time_sense"]

recon_grasp    = get_center(recon_grasp) 
recon_sense    = get_center(recon_sense)
recon_fs       = get_center(recon_fs) 

recon_grasp    = safe_normalize(recon_grasp) 
recon_sense    = safe_normalize(recon_sense) 
recon_fs       = safe_normalize(recon_fs) 

In [ ]:
reference_metrics = {
    'time_grasp':     time_grasp,
    'time_sense':     time_sense,
    'time_grasp_str': seconds_to_min_sec_format(total_time_grasp),
    'time_sense_str': seconds_to_min_sec_format(total_time_sense),
    'psnr_grasp':     mean_psnr(recon_grasp, recon_fs),
    'psnr_sense':     mean_psnr(recon_sense, recon_fs),
    'ssim_grasp':     mean_ssim(recon_grasp, recon_fs),
    'ssim_sense':     mean_ssim(recon_sense, recon_fs),
    'psnr_grasp_str':     str(round(float(mean_psnr(recon_grasp, recon_fs)),1)),
    'psnr_sense_str':     str(round(float(mean_psnr(recon_sense, recon_fs)),1)),
    'ssim_grasp_str':     str(round(float(mean_ssim(recon_grasp, recon_fs)),3)),
    'ssim_sense_str':     str(round(float(mean_ssim(recon_sense, recon_fs)),3)),
}

## 1.2 .- Models reconstructions

In [ ]:
target_columns = ['training_name', 'psnr', 'ssim', 'it', 'duration [min]', 'duration [s]']

### 1.2.1 .- ST-DIP

In [ ]:
stdip_parameters['path']               = base_folder + 'stDIP'
stdip_parameters['csv_path']           = stdip_parameters['path'] +  '/' + stdip_parameters['experiment_name'] + ".csv"
stdip_parameters['csv_path_summary']   = stdip_parameters['path'] +  '/' + stdip_parameters['experiment_name'] + "_summary.csv"
stdip_parameters['exp_folder_path']    = stdip_parameters['path'] +  '/' + dataset_name + '/' + stdip_parameters['experiment_name']
df_stdip                               = pd.read_csv(stdip_parameters['csv_path'] , delimiter=';')
df_stdip_summary                       = pd.read_csv(stdip_parameters['csv_path_summary'] , delimiter=';')

In [ ]:
stDIP_results                          = filter_and_get_columns(df_stdip, stdip_parameters['training_params'], target_columns)[0]

In [ ]:
stdip_parameters['best_recon_path']    = stdip_parameters['exp_folder_path'] +  '/' + stDIP_results['training_name'] + '/' + 'best_recon.npz'

In [ ]:
stDIP_results['recon']     = onp.load(stdip_parameters['best_recon_path'], allow_pickle=True)['best_recon']

### 1.2.2 .- TD-DIP

In [ ]:
tddip_parameters['path']               = base_folder + 'tdDIP'
tddip_parameters['csv_path']           = tddip_parameters['path'] +  '/' + tddip_parameters['experiment_name'] + ".csv"
tddip_parameters['csv_path_summary']   = tddip_parameters['path'] +  '/' + tddip_parameters['experiment_name'] + "_summary.csv"
tddip_parameters['exp_folder_path']    = tddip_parameters['path'] +  '/' + dataset_name + '/' + tddip_parameters['experiment_name']
df_tddip                               = pd.read_csv(tddip_parameters['csv_path'] , delimiter=';')
df_tddip_summary                       = pd.read_csv(tddip_parameters['csv_path_summary'] , delimiter=';')

In [ ]:
tdDIP_results                          = filter_and_get_columns(df_tddip, tddip_parameters['training_params'], target_columns)[0]

In [ ]:
tdDIP_summary_results                         =    filter_and_get_columns(df_tddip_summary, tddip_summray_parameters, ['duration [s]_mean'])[0]
tdDIP_results['total_recon_time [s]']         =    tdDIP_summary_results['duration [s]_mean'] * total_slices
tdDIP_results['total_recon_time [s]'], tdDIP_results['total_recon_time [s]']//60

In [ ]:
tddip_parameters['best_recon_path']    = tddip_parameters['exp_folder_path'] +  '/' + tdDIP_results['training_name'] + '/' + 'best_recon.npz'

In [ ]:
tdDIP_results['recon']     = onp.load(tddip_parameters['best_recon_path'], allow_pickle=True)['best_recon']

In [ ]:
stDIP_results['recon']     = safe_normalize(stDIP_results['recon'] ) 
tdDIP_results['recon']     = safe_normalize(tdDIP_results['recon'] ) 

# 2 .- Comparison of models

In [ ]:
comparison_folder = base_folder + 'comparison/'
if not os.path.exists(comparison_folder):
    os.makedirs(comparison_folder)

## 2.1 .- Parameters for visualiation

In [ ]:
add_psnr = False
add_ssim = False
add_time = True
gt_title    = r'$\mathbf{Ground Truth}$'
stdip_title = r'$\mathbf{ST\text{-}DIP}$'
tddip_title = r'$\mathbf{TD\text{-}DIP}$'
grasp_title = r'$\mathbf{GRASP}$'
sense_title = r'$\mathbf{SENSE}$'
if add_psnr:
    gt_title    += '\n '
    stdip_title += '\npsnr: ' + str(round(stDIP_results['psnr'],1))
    tddip_title += '\npsnr: ' + str(round(tdDIP_results['psnr'],1))
    grasp_title += '\npsnr: ' + reference_metrics['psnr_grasp_str']
    sense_title += '\npsnr: ' + reference_metrics['psnr_sense_str']

if add_ssim:
    gt_title    += '\n '
    stdip_title += '\nssim: ' + str(round(stDIP_results['ssim'],3))
    tddip_title += '\nssim: ' + str(round(tdDIP_results['ssim'],3))
    grasp_title += '\nssim: ' + reference_metrics['ssim_grasp_str']
    sense_title += '\nssim: ' + reference_metrics['ssim_sense_str']

if add_time:
    gt_title    += '\n '
    stdip_title += '\ntotal recon time: ' + seconds_to_min_sec_format(stDIP_results['duration [s]'])
    tddip_title += '\ntotal recon time: ' + seconds_to_min_sec_format(tdDIP_results['total_recon_time [s]'])
    grasp_title += '\ntotal recon time: ' + reference_metrics['time_grasp_str']
    sense_title += '\ntotal recon time: ' + reference_metrics['time_sense_str']

In [ ]:
stDIP_results['name'] = stdip_title
tdDIP_results['name'] = tddip_title
name_grasp            = grasp_title
name_sense            = sense_title

## 2.2 .- Plotting customization per volunteer

In [ ]:
stDIP_results['recon_trans'] = apply_transform(stDIP_results['recon'], dataset, volunteer, dict_trans)
tdDIP_results['recon_trans'] = apply_transform(tdDIP_results['recon'], dataset, volunteer, dict_trans)
recon_grasp_trans = apply_transform(recon_grasp, dataset, volunteer, dict_trans)
recon_sense_trans = apply_transform(recon_sense, dataset, volunteer, dict_trans)
if volunteer in ['FP', 'DP']:
    recon_fs_trans = apply_transform(recon_fs, dataset, volunteer, gt_trans)
else:
    recon_fs_trans = apply_transform(recon_fs, dataset, volunteer, dict_trans)

In [ ]:
matrix_list  = [recon_fs_trans, stDIP_results['recon_trans'], tdDIP_results['recon_trans'] , recon_grasp_trans, recon_sense_trans]
matrix_names = [gt_title, stDIP_results['name'], tdDIP_results['name'] , name_grasp , name_sense ]

crop_ns       = dict_plot[dataset][volunteer][slice_num]['crop_ns']
saturation    = dict_plot[dataset][volunteer][slice_num]['saturation']
frame = 0
print(crop_ns, saturation)

In [ ]:
fig, axs = full_halph_FOV_space_time(matrix_list, crop_ns=crop_ns, saturation=saturation, frame=frame, xy_proportion=(2,7))
for ax, title in zip(axs, matrix_names):
    ax[0].set_title(title)
fig.savefig( comparison_folder + 'slice' + str(slice_num) + ".png", dpi=300, bbox_inches='tight')

# 3 .- Get Sharpness

In [ ]:
from inrmri.image_processor import crop_2d_im 
from inrmri.sharpness import (
    radial_sharpness,
    extract_line_at_points,
    get_line_points_from_angle,
    find_percentile_idx, 
    get_distance
)

In [ ]:
recon_fs_trans.shape

In [ ]:
# parameters
center           = np.array([22, 22])  # coordenadas del centro del miocardio
radius_pixels    = 10                  # radio en pixeles del perfil 
selected_frame   = 0
angle            = np.pi * (-1/4)
im               = crop_2d_im(recon_grasp_trans[...,selected_frame], crop_ns)
vmax             = 0.6 * np.abs(im).max() 
im.shape

In [ ]:
def get_profile(image, frame, crop_ns):
    im                 = crop_2d_im(image[...,frame], crop_ns)
    vmax               = 0.6 * np.abs(im).max() 
    points, idx_center = get_line_points_from_angle(center, angle, im.shape)
    points             = points[idx_center-radius_pixels:idx_center,:] # mitad izquierda
    profile            = extract_line_at_points(im, points)
    return profile

In [ ]:
recon_grasp_trans.shape

In [ ]:
# recon fully-sampled 
fs_profile          = get_profile(recon_fs_trans, selected_frame, crop_ns)
stdip_profile       = get_profile(stDIP_results['recon_trans'], selected_frame, crop_ns)
tddip_profile       = get_profile(tdDIP_results['recon_trans'], selected_frame, crop_ns)
grasp_profile       = get_profile(recon_grasp_trans, selected_frame, crop_ns)
sense_profile       = get_profile(recon_sense_trans, selected_frame, crop_ns)
# stDIP
plt.plot(fs_profile, '.-', label= 'FS')
plt.plot(stdip_profile, '.-', label= 'ST-DIP')
plt.plot(tddip_profile, '.-', label= 'TD-DIP')
plt.plot(grasp_profile, '.-', label= 'GRASP')
plt.plot(sense_profile, '.-', label= 'SENSE')
plt.legend()

In [ ]:
im.shape, d, d_20_80

In [ ]:
plt.figure()
plt.subplot(211)
plt.imshow(np.abs(im), cmap='bone', vmax=vmax)
plt.plot(points[:,1],
        points[:,0],
        color='red'
        )
plt.subplot(212)
plt.plot(profile, '.-', color = 'red', label= 'profile')

p20_color = 'orange'
plt.hlines([p20], xmin=0, xmax=14, color=p20_color, label='p20', linestyles=(0,(1,3)))
plt.vlines([idx_20], ymin=profile.min(), ymax=profile.max(), color=p20_color, label='idx_20', linestyles=(0,(1,1)))

p80_color = 'blue'
plt.hlines([p80], xmin=0, xmax=14, color=p80_color, label='p80', linestyles=(0, (3,3,1,3)))
plt.vlines([idx_80], ymin=profile.min(), ymax=profile.max(), color=p80_color, label='idx_80', linestyles=(0, (3,1,1,1)))

plt.hlines([profile.min()], xmin=idx_20, xmax=idx_80, color='gray', linestyles='-')
plt.text(x=(idx_20 + idx_80)/2, y=profile.min() + 0.07 * (profile.max() - profile.min()), s='$d_{20, 80} = $' + f'{d_20_80}', ha='center', va='center', color='gray')

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('$d_{20,80} = ' f'{d_20_80}$, sharpness 1/d {1 / d:.3f}')
plt.tight_layout()